## Cirrhosis Patient Survival Prediction Menggunakan Algoritma Klasifikasi KNN

Pada proyek UTS ini akan menganalisi data menggunakan tahapan analisis pada data pasien sirosis hati menggunakan algoritma klasifikasi yang saya pilih yaitu **K-Nearest Neighbors (KNN)**.

### Tahapan Analisis:
1. Pemahaman Data
2. Preprocessing (Pemrosesan Awal)
3. Pemodelan: KNN
4. Evaluasi Model


### 1. Pemahaman Data
Pada dataset ini memiliki 418 baris dan 20 kolom, dengan informasi pasien sirosis hati. Teerdapat kolom-kolom yang penting yang menjadi label atau target kalasifikasi yang saya buat. 
- Status: 
kolom ini berisi
    - C: Censored (pasien masih hidup saat studi berakhir)
    - CL: Censored karena liver transplant
    - D: Death (pasien meninggal)

In [2]:
import pandas as pd

df = pd.read_csv('cirrhosis.csv')
df.head()

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 20 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   ID             418 non-null    int64  
 1   N_Days         418 non-null    int64  
 2   Status         418 non-null    object 
 3   Drug           312 non-null    object 
 4   Age            418 non-null    int64  
 5   Sex            418 non-null    object 
 6   Ascites        312 non-null    object 
 7   Hepatomegaly   312 non-null    object 
 8   Spiders        312 non-null    object 
 9   Edema          418 non-null    object 
 10  Bilirubin      418 non-null    float64
 11  Cholesterol    284 non-null    float64
 12  Albumin        418 non-null    float64
 13  Copper         310 non-null    float64
 14  Alk_Phos       312 non-null    float64
 15  SGOT           312 non-null    float64
 16  Tryglicerides  282 non-null    float64
 17  Platelets      407 non-null    float64
 18  Prothrombi

### 2. Rencana Preprocessing
Pada tahap ini agar data siap untuk diklasifikasi saya akan melakukan beberapa perubahan seperti 

1. Ubah target Status ke angka:

    - D → 1 (Meninggal)

    - C, CL → 0 (Tidak meninggal / censored)

2. Ubah kolom kategorikal jadi angka (Sex, Drug, Ascites, dll)

3. Tangani missing value:

    - Bisa dengan menghapus baris atau imputasi (mengisi nilai rata-rata/ modus)

4. Normalisasi fitur numerik

5. Pisahkan fitur dan label, lalu lakukan split data train/test

In [3]:
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.impute import SimpleImputer

# Salin data
data = df.copy()

# Ubah target ke numerik: D=1 (meninggal), C/CL=0 (tidak meninggal)
data['Status'] = data['Status'].replace({'D': 1, 'C': 0, 'CL': 0})

# Hapus kolom ID
data.drop(columns=['ID'], inplace=True)

# Label Encoding untuk kolom kategorikal
categorical_cols = data.select_dtypes(include=['object']).columns
le = LabelEncoder()
for col in categorical_cols:
    data[col] = le.fit_transform(data[col].astype(str))

# Tangani missing values dengan mean
imputer = SimpleImputer(strategy='mean')
data_imputed = pd.DataFrame(imputer.fit_transform(data), columns=data.columns)


C:\Users\Rypaldho Ridotua H\AppData\Local\Temp\ipykernel_11092\927418635.py:8: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data['Status'] = data['Status'].replace({'D': 1, 'C': 0, 'CL': 0})


In [4]:
from sklearn.model_selection import train_test_split

X = data_imputed.drop('Status', axis=1)
y = data_imputed['Status']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)


### 3. Permodelan
Saya akan melakukan permodelan data menggunakan KNN (K-Nearest Neighbors) 

In [5]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)


KNeighborsClassifier()

### 4. Evaluasi Model
Tahapan yang selanjutnya, setelah kita berhasil menjalankan model KNN pada dataset kita akan mengevaluasi model

In [6]:
from sklearn.metrics import confusion_matrix, classification_report

y_pred = knn.predict(X_test)

print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))


Confusion Matrix:
 [[44  4]
 [10 26]]

Classification Report:
               precision    recall  f1-score   support

         0.0       0.81      0.92      0.86        48
         1.0       0.87      0.72      0.79        36

    accuracy                           0.83        84
   macro avg       0.84      0.82      0.83        84
weighted avg       0.84      0.83      0.83        84



### Dari hasil evaluasi Confusion Matrix diperoleh: 

[[44,  4] 

[10, 26]]

Artinya ada sebanyak 44 pasien yang tidak meninggal dan di prediksi benar (True Negatives), sebanyak 26 pasien yang meninggal diprediksi benar (True Positives), ada 4 pasien yang tidak meninggal tapi di prediksi meninggal (False Positives) dan 10 pasien yang meninggal tapi diprediksi tidak meninggal (False Negatives) 

#### Clasification Report:

              precision    recall  f1-score   support

         0.0       0.81      0.92      0.86        48
         1.0       0.87      0.72      0.79        36

    accuracy                           0.83        84
    macro avg      0.84      0.82      0.83        84
    weighted avg   0.84      0.83      0.83        84

dari hasil akurasi keseluruhan diperoleh 83% yang berarti model ini cukup baik dalam memprediksi kematian pasien berdasarkan fitur fitur yang tersedia 

### Kesimpulan
Dalam analisis ini, saya menggunakan algoritma klasifikasi K-Nearest Neighbors (KNN) untuk memprediksi kelangsungan hidup pasien sirosis hati berdasarkan fitur medis dan demografis mereka.

Berdasarkan hasil evaluasi, model KNN memberikan performa yang cukup baik dengan akurasi sekitar 83%. Model ini berhasil mengenali pasien yang berisiko meninggal maupun yang tidak dengan keseimbangan yang cukup baik antara precision, recall, dan f1-score.

Beberapa poin penting yang dapat disimpulkan:

Fitur-fitur seperti bilirubin, albumin, ascites, dan edema tampaknya berperan penting dalam memprediksi status pasien.

Model masih memiliki tantangan dalam mengurangi kesalahan prediksi terhadap pasien yang meninggal (false negatives).

Dengan preprocessing yang tepat seperti imputasi missing value dan normalisasi, algoritma KNN cukup efektif diterapkan untuk data kesehatan seperti ini.

Model ini dapat menjadi dasar dalam pengembangan sistem pendukung keputusan medis, namun tidak boleh digunakan sebagai pengganti diagnosis profesional.

